## Basic Example

In [ ]:
import coiled
cluster = coiled.Cluster(n_workers=10)

from dask.distributed import Client
client = Client(cluster)
print('Dashboard:', client.dashboard_link)

In [6]:
import dask.dataframe as dd

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()


In [7]:
df.groupby("passenger_count").tip_amount.mean().compute()

passenger_count
0    2.122789
1    2.206790
2    2.214306
3    2.137775
4    2.023804
5    2.235441
6    2.221105
7    6.675962
8    7.111625
9    7.377822
Name: tip_amount, dtype: float64

## Dask Optuna

In [1]:
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [2]:
def objective(trial):
    # Load our dataset
    X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Get set of hyperparameters
    param = {
        "silent": 1,
        "objective": "binary:logistic",
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 9),
        "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),
    }

    # Train XGBoost model
    bst = xgb.train(param, dtrain)
    preds = bst.predict(dtest)

    # Compute and return model accuracy
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_test, pred_labels)
    return accuracy

In [3]:
from dask.distributed import Client
import coiled
import dask_optuna
import joblib
import optuna

In [4]:
# Create a Dask cluster with Coiled
# cluster = coiled.Cluster(n_workers=10, configuration="jrbourbeau/optuna")
cluster = coiled.Cluster(n_workers=10)
# Connect Dask to our cluster
client = Client(cluster)
print(f"Dask dashboard is available at {client.dashboard_link}")
client.wait_for_workers(10)

/Users/seanswe/Documents/projects/coiled/coiledvenv/lib/python3.8/site-packages/rich/live.py:216: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


/Users/seanswe/Documents/projects/coiled/coiledvenv/lib/python3.8/site-packages/distributed/client.py:1134: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| blosc   | None          | 1.10.2        | 1.10.2        |
| lz4     | None          | 3.1.3         | 3.1.3         |
| numpy   | 1.19.5        | 1.20.1        | 1.20.1        |
| python  | 3.8.2.final.0 | 3.8.8.final.0 | 3.8.8.final.0 |
+---------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Dask dashboard is available at http://ec2-13-56-227-222.us-west-1.compute.amazonaws.com:8787


In [ ]:
# Create Dask-compatible Optuna storage class
# storage = dask_optuna.DaskStorage()

In [5]:
# Run 500 optimizations trial on our cluster
# study = optuna.create_study(direction="maximize", storage=storage)
study = optuna.create_study(direction="maximize")
with joblib.parallel_backend("dask"):
    study.optimize(objective, n_trials=500, n_jobs=-1)

[I 2021-03-26 22:48:49,176] A new study created in memory with name: no-name-c9abd3f1-bca5-4f89-9254-20ba5a0dd7bb
[I 2021-03-26 22:48:49,445] Trial 0 finished with value: 0.9230769230769231 and parameters: {'booster': 'dart', 'lambda': 8.495630143695366e-06, 'alpha': 2.931077572706209e-07, 'max_depth': 2, 'eta': 1.2079292129676216e-06, 'gamma': 0.0010032136703249286, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.9230769230769231.
[I 2021-03-26 22:48:49,505] Trial 3 finished with value: 0.9230769230769231 and parameters: {'booster': 'gbtree', 'lambda': 9.3545816095663e-07, 'alpha': 1.2967563777175634e-07, 'max_depth': 3, 'eta': 0.019688728411571734, 'gamma': 0.0001126106397647235, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.9230769230769231.
[I 2021-03-26 22:48:49,570] Trial 1 finished with value: 0.958041958041958 and parameters: {'booster': 'dart', 'lambda': 1.5082280331925498e-07, 'alpha': 0.0032629689451655042, 'max_depth': 5, 'eta': 0.004895710461800676,

[I 2021-03-26 22:48:52,177] Trial 24 finished with value: 0.916083916083916 and parameters: {'booster': 'gbtree', 'lambda': 0.09173831666157344, 'alpha': 0.20582045648268357, 'max_depth': 4, 'eta': 9.417724390720238e-07, 'gamma': 0.012698392285380907, 'grow_policy': 'depthwise'}. Best is trial 9 with value: 0.9790209790209791.
[I 2021-03-26 22:48:52,246] Trial 26 finished with value: 0.8881118881118881 and parameters: {'booster': 'gbtree', 'lambda': 0.06280257264251757, 'alpha': 0.14000241874218605, 'max_depth': 1, 'eta': 0.0001398858398077257, 'gamma': 0.00011047572551633387, 'grow_policy': 'depthwise'}. Best is trial 9 with value: 0.9790209790209791.
[I 2021-03-26 22:48:52,259] Trial 25 finished with value: 0.916083916083916 and parameters: {'booster': 'gbtree', 'lambda': 0.04152860733950019, 'alpha': 8.70791609481295e-05, 'max_depth': 1, 'eta': 9.227704333702923e-05, 'gamma': 0.00012650872119071, 'grow_policy': 'depthwise'}. Best is trial 9 with value: 0.9790209790209791.
[I 2021-03

[I 2021-03-26 22:48:54,873] Trial 48 finished with value: 0.916083916083916 and parameters: {'booster': 'gbtree', 'lambda': 2.6087645071703645e-07, 'alpha': 1.1608087325777003e-06, 'max_depth': 7, 'eta': 0.12224282835303688, 'gamma': 1.4656629847382754e-07, 'grow_policy': 'lossguide'}. Best is trial 43 with value: 0.986013986013986.
[I 2021-03-26 22:48:54,914] Trial 49 finished with value: 0.9440559440559441 and parameters: {'booster': 'gbtree', 'lambda': 2.574666812638386e-07, 'alpha': 4.499706620445967e-06, 'max_depth': 7, 'eta': 0.2487053664456257, 'gamma': 3.7013007080438313e-07, 'grow_policy': 'lossguide'}. Best is trial 43 with value: 0.986013986013986.
[I 2021-03-26 22:48:54,983] Trial 50 finished with value: 0.9230769230769231 and parameters: {'booster': 'gbtree', 'lambda': 3.471255115639051e-07, 'alpha': 3.477234465023326e-06, 'max_depth': 7, 'eta': 0.0898953443258017, 'gamma': 2.817578405103303e-07, 'grow_policy': 'lossguide'}. Best is trial 43 with value: 0.986013986013986.


[I 2021-03-26 22:48:58,017] Trial 72 finished with value: 0.951048951048951 and parameters: {'booster': 'gbtree', 'lambda': 0.025694686331990412, 'alpha': 3.8626906074241216e-05, 'max_depth': 9, 'eta': 1.8782969052251257e-06, 'gamma': 7.790637275927843e-06, 'grow_policy': 'depthwise'}. Best is trial 43 with value: 0.986013986013986.
[I 2021-03-26 22:48:58,041] Trial 74 finished with value: 0.9790209790209791 and parameters: {'booster': 'gbtree', 'lambda': 0.016887493184880382, 'alpha': 0.000275869931478815, 'max_depth': 3, 'eta': 0.483877480865347, 'gamma': 2.1966515664818144e-05, 'grow_policy': 'depthwise'}. Best is trial 43 with value: 0.986013986013986.
[I 2021-03-26 22:48:58,052] Trial 73 finished with value: 0.951048951048951 and parameters: {'booster': 'gbtree', 'lambda': 1.7199100850899947e-08, 'alpha': 0.0002237494829661058, 'max_depth': 3, 'eta': 0.6084200749147415, 'gamma': 4.688841751493157e-06, 'grow_policy': 'depthwise'}. Best is trial 43 with value: 0.986013986013986.
[I 

[I 2021-03-26 22:49:00,658] Trial 94 finished with value: 0.951048951048951 and parameters: {'booster': 'gbtree', 'lambda': 0.11458205524385406, 'alpha': 0.0001922014272591023, 'max_depth': 9, 'eta': 0.8578522185611785, 'gamma': 0.00041305750391727215, 'grow_policy': 'depthwise'}. Best is trial 43 with value: 0.986013986013986.
[I 2021-03-26 22:49:00,665] Trial 97 finished with value: 0.958041958041958 and parameters: {'booster': 'gbtree', 'lambda': 1.2355727810384273e-06, 'alpha': 0.00015268950282849644, 'max_depth': 1, 'eta': 0.8689802030335795, 'gamma': 0.005147973684896544, 'grow_policy': 'depthwise'}. Best is trial 43 with value: 0.986013986013986.
[I 2021-03-26 22:49:00,741] Trial 98 finished with value: 0.958041958041958 and parameters: {'booster': 'gbtree', 'lambda': 1.192548179011542e-06, 'alpha': 0.00015017357217322623, 'max_depth': 9, 'eta': 0.9150843579111831, 'gamma': 3.1893213805209066e-05, 'grow_policy': 'depthwise'}. Best is trial 43 with value: 0.986013986013986.
[I 20

...

[I 2021-03-26 22:49:29,600] Trial 384 finished with value: 0.9300699300699301 and parameters: {'booster': 'gbtree', 'lambda': 0.004117326513977963, 'alpha': 9.191548504422866e-05, 'max_depth': 5, 'eta': 1.997792119302173e-05, 'gamma': 6.357071188549947e-07, 'grow_policy': 'lossguide'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:29,752] Trial 386 finished with value: 0.965034965034965 and parameters: {'booster': 'dart', 'lambda': 0.00014152691861467605, 'alpha': 8.833760079820633e-05, 'max_depth': 6, 'eta': 0.2695756968930638, 'gamma': 3.931807007662032e-07, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:29,798] Trial 385 finished with value: 0.958041958041958 and parameters: {'booster': 'dart', 'lambda': 0.000165886565916062, 'alpha': 6.818622277854075e-05, 'max_depth': 6, 'eta': 0.2761240775451039, 'gamma': 7.721374421946457e-06, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.
[I

[I 2021-03-26 22:49:33,157] Trial 408 finished with value: 0.9370629370629371 and parameters: {'booster': 'gbtree', 'lambda': 0.055641374214030634, 'alpha': 1.8095734165412146e-06, 'max_depth': 3, 'eta': 0.6417238824901913, 'gamma': 2.020365752006436e-06, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:33,271] Trial 409 finished with value: 0.916083916083916 and parameters: {'booster': 'gbtree', 'lambda': 0.00016792334238687728, 'alpha': 0.03670609753918697, 'max_depth': 4, 'eta': 0.00018103315369883438, 'gamma': 7.31756665168853e-05, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:33,420] Trial 410 finished with value: 0.9300699300699301 and parameters: {'booster': 'gbtree', 'lambda': 0.0021335651275090767, 'alpha': 0.07668976555433585, 'max_depth': 5, 'eta': 0.5997322347030344, 'gamma': 8.20682645225174e-07, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.


[I 2021-03-26 22:49:37,042] Trial 432 finished with value: 0.965034965034965 and parameters: {'booster': 'gbtree', 'lambda': 0.05067275260979878, 'alpha': 0.00015432621756598412, 'max_depth': 6, 'eta': 0.9638815387982448, 'gamma': 9.733244026415424e-07, 'grow_policy': 'lossguide'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:37,144] Trial 433 finished with value: 0.8951048951048951 and parameters: {'booster': 'gbtree', 'lambda': 0.04737627907846892, 'alpha': 6.534751031418121e-05, 'max_depth': 6, 'eta': 2.3877927900227424e-05, 'gamma': 8.097056834782895e-07, 'grow_policy': 'lossguide'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:37,235] Trial 434 finished with value: 0.9230769230769231 and parameters: {'booster': 'dart', 'lambda': 0.035797165689608064, 'alpha': 0.00011214125175033015, 'max_depth': 6, 'eta': 1.8554031551799353e-05, 'gamma': 6.754446143159087e-07, 'grow_policy': 'lossguide'}. Best is trial 244 with value: 0.9930069930069

[I 2021-03-26 22:49:40,411] Trial 456 finished with value: 0.9440559440559441 and parameters: {'booster': 'gbtree', 'lambda': 0.07494455033625316, 'alpha': 1.4006066907177197e-06, 'max_depth': 8, 'eta': 3.4936877837415645e-05, 'gamma': 7.701408442510065e-07, 'grow_policy': 'lossguide'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:40,462] Trial 457 finished with value: 0.916083916083916 and parameters: {'booster': 'gbtree', 'lambda': 0.3195173528459447, 'alpha': 3.482737829532528e-06, 'max_depth': 8, 'eta': 4.612942912798069e-05, 'gamma': 1.0541258691355135e-06, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:40,656] Trial 458 finished with value: 0.9370629370629371 and parameters: {'booster': 'gbtree', 'lambda': 0.07101037829877103, 'alpha': 5.045448146872642e-05, 'max_depth': 5, 'eta': 3.516126451478633e-05, 'gamma': 1.0428332528445949e-06, 'grow_policy': 'lossguide'}. Best is trial 244 with value: 0.9930069930

[I 2021-03-26 22:49:44,233] Trial 480 finished with value: 0.9440559440559441 and parameters: {'booster': 'gbtree', 'lambda': 0.08947318232314294, 'alpha': 4.9286974623028443e-05, 'max_depth': 6, 'eta': 0.32023072346459486, 'gamma': 4.2079512948009366e-05, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:44,334] Trial 481 finished with value: 0.9230769230769231 and parameters: {'booster': 'gbtree', 'lambda': 0.05255500729046451, 'alpha': 5.767223594036301e-05, 'max_depth': 6, 'eta': 0.3348036014099874, 'gamma': 5.393619490793035e-05, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.
[I 2021-03-26 22:49:44,513] Trial 482 finished with value: 0.9300699300699301 and parameters: {'booster': 'dart', 'lambda': 0.13735112222666865, 'alpha': 3.016804464390958e-06, 'max_depth': 7, 'eta': 0.11394617391499343, 'gamma': 5.314559654504319e-05, 'grow_policy': 'depthwise'}. Best is trial 244 with value: 0.993006993006993.
[I

In [ ]:
study.trials_dataframe()